##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/custom_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/custom_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Keras custom callbacks
A custom callback is a powerful tool to customize the behavior of a Keras model during training, evaluation, or inference, including reading/changing the Keras model. Examples include `tf.keras.callbacks.TensorBoard` where the training progress and results can be exported and visualized with TensorBoard, or `tf.keras.callbacks.ModelCheckpoint` where the model is automatically saved during training, and more. In this guide, you will learn what Keras callback is, when it will be called, what it can do, and how you can build your own. Towards the end of this guide, there will be demos of creating a couple of simple callback applications to get you started on your custom callback.

## Setup

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

C:\Users\Eugenio\Anaconda3\envs\tf2\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Eugenio\Anaconda3\envs\tf2\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Eugenio\Anaconda3\envs\tf2\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Eugenio\Anaconda3\envs\tf2\lib\site-package

## Introduction to Keras callbacks
In Keras, `Callback` is a python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. You can pass a list of callbacks (as the keyword argument `callbacks`) to any of `tf.keras.Model.fit()`, `tf.keras.Model.evaluate()`, and `tf.keras.Model.predict()` methods. The methods of the callbacks will then be called at different stages of training/evaluating/inference.

To get started, let's import tensorflow and define a simple Sequential Keras model:

In [3]:
# Define the Keras model to add callbacks to
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

Then, load the MNIST data for training and testing from Keras datasets API:

In [4]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

Now, define a simple custom callback to track the start and end of every batch of data. During those calls, it prints the index of the current batch.

In [5]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_begin(self, batch, logs=None):
    print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_test_batch_end(self, batch, logs=None):
    print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

Providing a callback to model methods such as `tf.keras.Model.fit()` ensures the methods are called at those stages:

In [6]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

Training: batch 0 begins at 07:42:33.551182
Training: batch 0 ends at 07:42:34.206021
Training: batch 1 begins at 07:42:34.207067
Training: batch 1 ends at 07:42:34.209905
Training: batch 2 begins at 07:42:34.209905
Training: batch 2 ends at 07:42:34.210691
Training: batch 3 begins at 07:42:34.210691
Training: batch 3 ends at 07:42:34.215599
Training: batch 4 begins at 07:42:34.216084
Training: batch 4 ends at 07:42:34.218823


## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
#### [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
#### [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
#### [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.


In [7]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5,
          callbacks=[MyCustomCallback()])

Evaluating: batch 0 begins at 07:42:50.638483
Evaluating: batch 0 ends at 07:42:50.735898
Evaluating: batch 1 begins at 07:42:50.736902
Evaluating: batch 1 ends at 07:42:50.740083
Evaluating: batch 2 begins at 07:42:50.740083
Evaluating: batch 2 ends at 07:42:50.741085
Evaluating: batch 3 begins at 07:42:50.741085
Evaluating: batch 3 ends at 07:42:50.741085
Evaluating: batch 4 begins at 07:42:50.741085
Evaluating: batch 4 ends at 07:42:50.745635


## An overview of callback methods


### Common methods for training/testing/predicting
For training, testing, and predicting, following methods are provided to be overridden.
#### `on_(train|test|predict)_begin(self, logs=None)`
Called at the beginning of `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_end(self, logs=None)`
Called at the end of `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`
Called right before processing a batch during training/testing/predicting. Within this method, __`logs` is a dict with `batch` and `size` available keys__, representing the __current batch number and the size of the batch__.
#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`
Called at the end of training/testing/predicting a batch. Within this method, `logs` is a __dict containing the stateful metrics result__.

### Training specific methods
In addition, for training, following are provided.
#### on_epoch_begin(self, epoch, logs=None)
Called at the beginning of an epoch during training.
#### on_epoch_end(self, epoch, logs=None)
Called at the end of an epoch during training.


### Usage of `logs` dict
The `logs` dict contains the loss value, and all the metrics at the end of a batch or epoch. Example includes the loss and mean absolute error.

In [8]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

  def on_train_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   32.60.
For batch 1, loss is  977.06.
For batch 2, loss is   26.34.
For batch 3, loss is    9.67.
For batch 4, loss is    8.72.
The average loss for epoch 0 is  210.88 and mean absolute error is    8.68.
For batch 0, loss is    7.02.
For batch 1, loss is    6.24.
For batch 2, loss is    7.08.
For batch 3, loss is    7.00.
For batch 4, loss is    8.16.
The average loss for epoch 1 is    7.10 and mean absolute error is    2.19.
For batch 0, loss is    6.65.
For batch 1, loss is    4.38.
For batch 2, loss is    5.21.
For batch 3, loss is    6.02.
For batch 4, loss is    5.20.
The average loss for epoch 2 is    5.49 and mean absolute error is    1.94.


Similarly, one can provide callbacks in `evaluate()` calls.

In [9]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is    5.60.
For batch 1, loss is    3.91.
For batch 2, loss is    4.94.
For batch 3, loss is    5.03.
For batch 4, loss is    5.88.
For batch 5, loss is    4.70.
For batch 6, loss is    4.75.
For batch 7, loss is    4.44.
For batch 8, loss is    4.73.
For batch 9, loss is    5.69.
For batch 10, loss is    5.49.
For batch 11, loss is    5.58.
For batch 12, loss is    5.79.
For batch 13, loss is    7.22.
For batch 14, loss is    4.87.
For batch 15, loss is    5.16.
For batch 16, loss is    6.29.
For batch 17, loss is    6.54.
For batch 18, loss is    6.31.
For batch 19, loss is    4.51.


## Examples of Keras callback applications
The following section will guide you through creating simple Callback applications.

### Early stopping at minimum loss
First example showcases the creation of a `Callback` that stops the Keras training when the minimum of loss has been reached __by mutating the attribute `model.stop_training` (boolean)__. Optionally, the user can provide an argument `patience` to specfify how many epochs the training should wait before it eventually stops.

`tf.keras.callbacks.EarlyStopping` provides a more complete and general implementation.

In [10]:
import numpy as np

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        #Paramos el entrenamiento
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

In [11]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

For batch 0, loss is   22.00.
For batch 1, loss is  824.54.
For batch 2, loss is   20.08.
For batch 3, loss is   12.31.
For batch 4, loss is    8.96.
The average loss for epoch 0 is  177.58 and mean absolute error is    8.02.
For batch 0, loss is    6.24.
For batch 1, loss is    5.54.
For batch 2, loss is    4.51.
For batch 3, loss is    5.26.
For batch 4, loss is    8.51.
The average loss for epoch 1 is    6.01 and mean absolute error is    2.00.
For batch 0, loss is    6.76.
For batch 1, loss is    5.01.
For batch 2, loss is    5.96.
For batch 3, loss is    4.84.
For batch 4, loss is    5.34.
The average loss for epoch 2 is    5.58 and mean absolute error is    1.92.
For batch 0, loss is    5.15.
For batch 1, loss is    4.82.
For batch 2, loss is    9.00.
For batch 3, loss is   28.72.
For batch 4, loss is  114.01.
The average loss for epoch 3 is   32.34 and mean absolute error is    4.14.
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


### Learning rate scheduling

One thing that is commonly done in model training is changing the learning rate as more epochs have passed. __Keras backend exposes get_value api which can be used to set the variables__. In this example, we're showing how a custom Callback can be used to dymanically change the learning rate.

Note: this is just an example implementation see `callbacks.LearningRateScheduler` and `keras.optimizers.schedules` for more general implementations.

In [12]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

  def __init__(self, schedule):
    super(LearningRateScheduler, self).__init__()
    self.schedule = schedule

  def on_epoch_begin(self, epoch, logs=None):
    if not hasattr(self.model.optimizer, 'lr'):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    #Podemos interactuar con el modelo. Aqui recuperamos un valor
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
    # Call schedule function to get the scheduled learning rate.
    scheduled_lr = self.schedule(epoch, lr)
    # Set the value back to the optimizer before this epoch starts
    #Podemos interactuar con el modelo. Aqui actualizamos un valor
    tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    print('\nEpoch %05d: Learning rate is %6.4f.' % (epoch, scheduled_lr))

In [13]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
  """Helper function to retrieve the scheduled learning rate based on epoch."""
  if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
    return lr
  for i in range(len(LR_SCHEDULE)):
    if epoch == LR_SCHEDULE[i][0]:
      return LR_SCHEDULE[i][1]
  return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   17.97.
For batch 1, loss is 1061.05.
For batch 2, loss is   20.94.
For batch 3, loss is    9.75.
For batch 4, loss is    6.84.
The average loss for epoch 0 is  223.31 and mean absolute error is    8.41.

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    5.55.
For batch 1, loss is    7.26.
For batch 2, loss is    5.09.
For batch 3, loss is    6.07.
For batch 4, loss is    5.28.
The average loss for epoch 1 is    5.85 and mean absolute error is    1.96.

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    5.50.
For batch 1, loss is    6.10.
For batch 2, loss is    5.71.
For batch 3, loss is    8.08.
For batch 4, loss is    5.85.
The average loss for epoch 2 is    6.25 and mean absolute error is    2.01.

Epoch 00003: Learning rate is 0.0500.
For batch 0, loss is    5.73.
For batch 1, loss is    4.33.
For batch 2, loss is    3.53.
For batch 3, loss is    5.04.
For batch 4, loss is    5.11.
The average loss

### Standard Keras callbacks
Be sure to check out the existing Keras callbacks by [visiting the api doc](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks). Applications include logging to CSV, saving the model, visualizing on TensorBoard and a lot more.